### 1. Import bibliotek.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from scipy.stats import spearmanr
from pyxirr import xirr
from sklearn.metrics import r2_score, make_scorer
from sklearnex.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.ERROR)

### 2. Dodatkowe funkcje.

In [2]:
import logging
from typing import Union, Optional

# Konfiguracja loggera
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def safe_divide(numerator: Union[pd.Series, float, int], 
                denominator: Union[pd.Series, float, int], 
                fill_value: Optional[float] = np.nan) -> Union[pd.Series, float]:
    """
    Bezpieczne dzielenie z obsługą dzielenia przez zero i wartości null.
    
    Parameters:
    numerator: licznik
    denominator: mianownik
    fill_value: wartość zwracana w przypadku dzielenia przez zero lub null
    
    Returns:
    Wynik dzielenia lub fill_value w przypadku błędu
    """
    try:
        # Konwersja do pandas Series jeśli potrzebna
        if not isinstance(numerator, pd.Series):
            numerator = pd.Series([numerator])
        if not isinstance(denominator, pd.Series):
            denominator = pd.Series([denominator])
            
        # Sprawdzenie czy mianownik jest różny od zera i nie jest null
        mask = (denominator != 0) & (~denominator.isna()) & (~numerator.isna())
        
        result = pd.Series(index=numerator.index, dtype=float)
        result.loc[mask] = numerator.loc[mask] / denominator.loc[mask]
        result.loc[~mask] = fill_value
        
        return result if len(result) > 1 else result.iloc[0]
        
    except Exception as e:
        logger.warning(f"Błąd w safe_divide: {e}")
        return fill_value

def validate_dataframe(df: pd.DataFrame) -> bool:
    """
    Walidacja DataFrame pod kątem wymaganych kolumn.
    """
    required_columns = [
        'total_current_assets', 'total_current_liabilities', 'cash_and_cash_equivalents',
        'total_debt', 'total_assets', 'total_shareholder_equity', 'net_income',
        'revenue', 'operating_income', 'weighted_average_shares'
    ]
    
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        logger.error(f"Brakuje wymaganych kolumn: {missing_columns}")
        return False
        
    return True

def calculate_financial_ratios(df: pd.DataFrame) -> pd.DataFrame:
    """
    Oblicza wskaźniki analizy fundamentalnej dla danego DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame zawierający dane finansowe.

    Returns:
    pd.DataFrame: DataFrame z dodanymi kolumnami wskaźników.
    
    Raises:
    ValueError: gdy DataFrame nie zawiera wymaganych kolumn
    TypeError: gdy df nie jest DataFrame
    """
    
    # Walidacja danych wejściowych
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Argument 'df' musi być typu pandas.DataFrame")
    
    if df.empty:
        logger.warning("DataFrame jest pusty")
        return df.copy()
    
    if not validate_dataframe(df):
        raise ValueError("DataFrame nie zawiera wszystkich wymaganych kolumn")
    
    # Tworzenie kopii DataFrame, aby nie modyfikować oryginału
    result_df = df.copy()
    
    try:
        # Obsługa brakujących kolumn - wypełnienie zerami lub NaN
        optional_columns = {
            'short_term_investments': 0,
            'accounts_receivable': 0,
            'inventory': 0,
            'cost_of_goods_sold': np.nan,
            'gross_profit': np.nan,
            'ebitda': np.nan,
            'long_term_debt': 0,
            'interest_expense': 0,
            'goodwill': 0,
            'intangible_assets': 0,
            'retained_earnings': np.nan,
            'accumulated_other_comprehensive_income': 0,
            'income_before_tax': np.nan,
            'accounts_payable': 0,
            'eps': np.nan,
            'adj_close': np.nan,
            'short_term_debt': 0,
            'net_debt': np.nan
        }
        
        for col, default_value in optional_columns.items():
            if col not in result_df.columns:
                result_df[col] = default_value
                logger.info(f"Dodano brakującą kolumnę '{col}' z wartością domyślną: {default_value}")

        # 1. Wskaźniki płynności
        result_df['current_ratio'] = safe_divide(
            result_df['total_current_assets'], 
            result_df['total_current_liabilities']
        )
        
        result_df['quick_ratio'] = safe_divide(
            result_df['cash_and_cash_equivalents'] + 
            result_df['short_term_investments'] + 
            result_df['accounts_receivable'],
            result_df['total_current_liabilities']
        )
        
        result_df['cash_ratio'] = safe_divide(
            result_df['cash_and_cash_equivalents'], 
            result_df['total_current_liabilities']
        )

        # 2. Wskaźniki zadłużenia
        result_df['debt_to_equity'] = safe_divide(
            result_df['total_debt'], 
            result_df['total_shareholder_equity']
        )
        
        result_df['debt_to_assets'] = safe_divide(
            result_df['total_debt'], 
            result_df['total_assets']
        )
        
        result_df['net_debt_to_ebitda'] = safe_divide(
            result_df['net_debt'], 
            result_df['ebitda']
        )

        # 3. Wskaźniki rentowności
        result_df['ROA'] = safe_divide(
            result_df['net_income'], 
            result_df['total_assets']
        )
        
        result_df['ROE'] = safe_divide(
            result_df['net_income'], 
            result_df['total_shareholder_equity']
        )
        
        # Poprawka ROIC - używamy NOPAT zamiast operating_income
        result_df['ROIC'] = safe_divide(
            result_df['operating_income'], 
            result_df['total_debt'] + result_df['total_shareholder_equity']
        )

        # 4. Wskaźniki efektywności operacyjnej
        result_df['asset_turnover'] = safe_divide(
            result_df['revenue'], 
            result_df['total_assets']
        )
        
        result_df['inventory_turnover'] = safe_divide(
            result_df['cost_of_goods_sold'], 
            result_df['inventory']
        )
        
        result_df['receivables_turnover'] = safe_divide(
            result_df['revenue'], 
            result_df['accounts_receivable']
        )

        # 5. Wskaźniki marżowe
        result_df['gross_margin'] = safe_divide(
            result_df['gross_profit'], 
            result_df['revenue']
        )
        
        result_df['operating_margin'] = safe_divide(
            result_df['operating_income'], 
            result_df['revenue']
        )
        
        result_df['net_margin'] = safe_divide(
            result_df['net_income'], 
            result_df['revenue']
        )
        
        result_df['ebitda_margin'] = safe_divide(
            result_df['ebitda'], 
            result_df['revenue']
        )

        # 6. Kapitał obrotowy
        result_df['working_capital'] = (
            result_df['total_current_assets'] - 
            result_df['total_current_liabilities']
        )

        # 7. Wskaźniki kapitału własnego
        result_df['book_value_per_share'] = safe_divide(
            result_df['total_shareholder_equity'], 
            result_df['weighted_average_shares']
        )

        # 8. Wskaźniki zadłużenia długoterminowego
        result_df['long_term_debt_to_equity'] = safe_divide(
            result_df['long_term_debt'], 
            result_df['total_shareholder_equity']
        )

        # 9. Wskaźnik pokrycia odsetek
        result_df['interest_coverage_ratio'] = safe_divide(
            result_df['operating_income'], 
            result_df['interest_expense']
        )

        # 10. Aktywa niematerialne
        result_df['goodwill_to_assets'] = safe_divide(
            result_df['goodwill'], 
            result_df['total_assets']
        )
        
        result_df['intangible_assets_to_assets'] = safe_divide(
            result_df['intangible_assets'], 
            result_df['total_assets']
        )

        # 11. Wskaźnik kapitału własnego
        result_df['equity_ratio'] = safe_divide(
            result_df['total_shareholder_equity'], 
            result_df['total_assets']
        )

        # 12. ROCE
        result_df['ROCE'] = safe_divide(
            result_df['operating_income'],
            result_df['total_assets'] - result_df['total_current_liabilities']
        )

        # 13. Retention Ratio
        result_df['retention_ratio'] = safe_divide(
            result_df['retained_earnings'], 
            result_df['net_income']
        )

        # 14. Comprehensive Income Ratio
        result_df['comprehensive_income_ratio'] = safe_divide(
            result_df['accumulated_other_comprehensive_income'], 
            result_df['net_income']
        )

        # 15. Interest Burden Ratio
        result_df['interest_burden_ratio'] = safe_divide(
            result_df['income_before_tax'], 
            result_df['operating_income']
        )

        # 16. Days Sales Outstanding (DSO)
        result_df['DSO'] = safe_divide(
            result_df['accounts_receivable'], 
            result_df['revenue']
        ) * 365

        # 17. Days Inventory Outstanding (DIO)
        result_df['DIO'] = safe_divide(
            result_df['inventory'], 
            result_df['cost_of_goods_sold']
        ) * 365

        # 18. Days Payable Outstanding (DPO)
        result_df['DPO'] = safe_divide(
            result_df['accounts_payable'], 
            result_df['cost_of_goods_sold']
        ) * 365

        # 19. Cash Conversion Cycle (CCC)
        result_df['cash_conversion_cycle'] = (
            result_df['DSO'] + result_df['DIO'] - result_df['DPO']
        )

        # 20. Wskaźniki Graham'a i wyceny
        # Bezpieczne obliczenie graham_number z obsługą wartości ujemnych
        eps_bvps_product = result_df['eps'] * result_df['book_value_per_share']
        result_df['graham_number'] = np.where(
            (eps_bvps_product > 0) & (~eps_bvps_product.isna()),
            np.sqrt(22.5 * eps_bvps_product),
            np.nan
        )
        
        result_df['price_to_earnings'] = safe_divide(
            result_df['adj_close'], 
            result_df['eps']
        )
        
        result_df['price_to_book'] = safe_divide(
            result_df['adj_close'], 
            result_df['book_value_per_share']
        )
        
        result_df['graham_number_vs_price'] = safe_divide(
            result_df['graham_number'], 
            result_df['adj_close']
        )
        
        # Usunięcie duplikatu debt_to_assets (już obliczone wcześniej)
        
        # Poprawka price_to_sales
        revenue_per_share = safe_divide(
            result_df['revenue'], 
            result_df['weighted_average_shares']
        )
        result_df['price_to_sales'] = safe_divide(
            result_df['adj_close'], 
            revenue_per_share
        )
        
        result_df['market_cap'] = (
            result_df['adj_close'] * result_df['weighted_average_shares']
        )
        
        result_df['pe_pb_product'] = (
            result_df['price_to_earnings'] * result_df['price_to_book']
        )
        
        result_df['total_debt_calculated'] = (
            result_df['short_term_debt'] + result_df['long_term_debt']
        )
        
        # Poprawka nazwy kolumny (typo)
        result_df['market_cap_vs_total_debt_calculated'] = safe_divide(
            result_df['market_cap'], 
            result_df['total_debt_calculated']
        )

        # 21. Wskaźniki "magicznej formuły"
        result_df['invested_capital'] = (
            result_df['total_assets'] - result_df['total_current_liabilities']
        )
        
        # Uproszczone obliczenie ROIC z lepszą obsługą błędów
        result_df['roic_magic_formula'] = safe_divide(
            result_df['operating_income'],
            result_df['invested_capital']
        )
        
        # Bezpieczne obliczenie net_debt
        if 'net_debt' not in result_df.columns or result_df['net_debt'].isna().all():
            result_df['net_debt'] = (
                result_df['total_debt'] - result_df['cash_and_cash_equivalents']
            )
        
        result_df['enterprise_value'] = (
            result_df['market_cap'] + result_df['net_debt']
        )
        
        result_df['earnings_yield'] = safe_divide(
            result_df['operating_income'],
            result_df['enterprise_value']
        )
        
        logger.info("Pomyślnie obliczono wszystkie wskaźniki finansowe")
        return result_df
        
    except Exception as e:
        logger.error(f"Błąd podczas obliczania wskaźników finansowych: {e}")
        raise


def calculate_portfolio_xirr(df, X, pred, best_thresh, label, label_length=12, investment_amount=1000):
    stocks = df.loc[X.index].copy()
    max_buy_date = (stocks['date'].max().to_timestamp(how='end') - pd.DateOffset(months=label_length))
    stocks['score'] = pred
    stocks['end_date'] = stocks[f'{label}_event_date'].astype(str)
    stocks['end_adj_close'] = stocks.adj_close + (stocks.adj_close * (stocks[f'{label}_pct_change']/100))
    cols = ['date', 'end_date', 'ticker', f'{label}_pct_change', 'adj_close', 'end_adj_close']
    stocks = stocks.loc[stocks['score'] > best_thresh]
    stocks = stocks.loc[stocks.groupby('date')['score'].idxmax()][cols]
    stocks['date'] = stocks['date'].astype(str)
    stocks['date'] = pd.PeriodIndex(stocks['date'], freq='Q').to_timestamp(how='end').date.astype(str)
    unique_dates = np.unique(stocks['date'].tolist() + stocks['end_date'].tolist())
    total_amount_invested = 0
    capital = 0
    wallet = {}
    log = []

    for i, todays_date in enumerate(unique_dates):
        # 1. Sprzedaż akcji (jeśli możliwa).
        stock_to_sell = wallet.get(todays_date)  # pobieram akcje do sprzedania
        if stock_to_sell is not None:  # gdy są jakieś akcje do sprzedania
            # 1.1. Pobieram akcje.
            stock_ticker = stock_to_sell['ticker']
            stock_buy_price = stock_to_sell['buy_price']
            stock_sell_price = stock_to_sell['sell_price']
            stock_num_of_shares = stock_to_sell['num_of_shares']

            # 1.2. Aktualizuję stan konta.
            total_amount = stock_num_of_shares * stock_sell_price
            capital = capital + total_amount

            # 1.3. Usuwam akcje z portfela.
            del wallet[todays_date]

            # dodanie loga
            log.append([todays_date, 'sprzedaż', stock_ticker, stock_sell_price, stock_num_of_shares, total_amount, capital-total_amount, capital])

        # 2. Kupno akcji.
        if (todays_date in stocks['date'].tolist()) & (pd.to_datetime(todays_date) <= max_buy_date):
            capital += investment_amount  # dodaję kapitał
            total_amount_invested += investment_amount

            # 2.1. Pobieram akcje.
            stock_to_buy = stocks[stocks['date'] == todays_date]
            stock_ticker = stock_to_buy['ticker'].values[0]
            stock_buy_price = stock_to_buy['adj_close'].values[0]
            stock_sell_price = stock_to_buy['end_adj_close'].values[0]
            stock_sell_date = stock_to_buy['end_date'].values[0]
            stock_num_of_shares = int(np.floor(capital/stock_buy_price))

            if stock_num_of_shares > 0:
                # 2.2. Aktualizuję stan konta.
                total_amount = stock_num_of_shares * stock_buy_price
                capital = capital - total_amount

                # 2.3. Dodaje akcje do portfela.
                wallet[stock_sell_date] = {
                    'ticker': stock_ticker,
                    'buy_price': stock_buy_price,
                    'sell_price': stock_sell_price,
                    'num_of_shares': stock_num_of_shares,
                    'buy_date': todays_date}

                # dodanie loga
                log.append([todays_date, 'kupno', stock_ticker, stock_buy_price, stock_num_of_shares, -total_amount, capital+total_amount, capital])

    log = pd.DataFrame(log, columns=['data', 'operacja', 'ticker', 'cena', 'liczba_sztuk', 'kwota_calkowita', 'stan_konta_przed', 'stan_konta_po'])
    if log.empty:
        srednioroczny_zwrot = 0
    else:
        log['data'] = pd.to_datetime(log['data'])
        xirr_value = xirr(log[['data', 'kwota_calkowita']])
        if xirr_value is not None:
            srednioroczny_zwrot = np.round(xirr_value * 100, 2)
        else:
            srednioroczny_zwrot = 0.0
    profit = capital - total_amount_invested
    return {
        'log': log,
        'xirr_percent': srednioroczny_zwrot,
        'total_amount_invested': total_amount_invested,
        'final_capital': capital,
        'profit': profit
    }

### 3. Wczytanie danych.

In [83]:
df = pd.read_feather('../data/raw/label_1000_100_250_clean.feather')

In [84]:
df = calculate_financial_ratios(df)

INFO:__main__:Pomyślnie obliczono wszystkie wskaźniki finansowe


In [85]:
target = 'label_1000_100_250_pct_change'

In [86]:
df = df[df['date'] != '2024Q3']

In [87]:
df['y'] = df.groupby('date')[target].transform(
    lambda x: pd.qcut(x, q=50, labels=False)
)

In [88]:
target = 'y'

In [89]:
cols = df.isnull().sum().sort_values(ascending=False)[df.isnull().sum().sort_values(ascending=False)<300].index.tolist()

In [90]:
df = df.dropna(subset=cols)

In [91]:
features_to_model = [col for col in cols if col not in ['y', 'date', 'ticker', 'industry', 'sector', 'label_1000_100_250_days_to_event', 'label_1000_100_250_target', 'label_1000_100_250_event_date', 'label_1000_100_250_pct_change', 'label_1000_100_250_final_price', 'adj_close']]

In [92]:
tr = df[df.date <= '2013Q3']
va = df[(df.date > '2013Q3') & (df.date <= '2017Q3')]
te = df[df.date > '2017Q3']

In [93]:
tr = tr.dropna(subset=cols)
va = va.dropna(subset=cols)
te = te.dropna(subset=cols)

In [124]:
features_to_model = ['book_value_per_share', 'price_to_book', 'price_to_sales', 'eps']

In [125]:
# model = ExtraTreesRegressor(n_estimators=5000, max_depth=3, min_samples_leaf=0.05)
model = RandomForestRegressor(n_estimators=5000, max_depth=3, min_samples_leaf=0.05)
model.fit(tr[features_to_model], tr['y'])

INFO:sklearnex: sklearn.ensemble.RandomForestRegressor.fit: running accelerated version on CPU
INFO:sklearnex:sklearn.ensemble.RandomForestRegressor.fit: running accelerated version on CPU


RandomForestRegressor(max_depth=3, min_samples_leaf=0.05, n_estimators=5000)

In [126]:
pred_tr = model.predict(tr[features_to_model])
pred_va = model.predict(va[features_to_model])
pred_te = model.predict(te[features_to_model])

INFO:sklearnex: sklearn.ensemble.RandomForestRegressor.predict: running accelerated version on CPU
INFO:sklearnex:sklearn.ensemble.RandomForestRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestRegressor.predict: running accelerated version on CPU
INFO:sklearnex:sklearn.ensemble.RandomForestRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.ensemble.RandomForestRegressor.predict: running accelerated version on CPU
INFO:sklearnex:sklearn.ensemble.RandomForestRegressor.predict: running accelerated version on CPU


In [127]:
score_tr = r2_score(tr[target], pred_tr)
score_va = r2_score(va[target], pred_va)
score_te = r2_score(te[target], pred_te)

corr_tr, p_value = spearmanr(tr[target], pred_tr)
corr_va, p_value = spearmanr(va[target], pred_va)
corr_te, p_value = spearmanr(te[target], pred_te)

print('CORR TR: {}'.format(np.round(corr_tr, 3)))
print('CORR VA: {}'.format(np.round(corr_va, 3)))
print('CORR TE: {}'.format(np.round(corr_te, 3)))

CORR TR: 0.228
CORR VA: 0.01
CORR TE: 0.027


In [128]:
summary_tr = calculate_portfolio_xirr(df, tr, pred_tr, -100, 'label_1000_100_250')
xirr_tr = summary_tr['xirr_percent']
print('XIRR TR: {}%'.format(xirr_tr))

summary_va = calculate_portfolio_xirr(df, va, pred_va, -100, 'label_1000_100_250')
xirr_va = summary_va['xirr_percent']
print('XIRR VA: {}%'.format(xirr_va))

summary_te = calculate_portfolio_xirr(df, te, pred_te, -100, 'label_1000_100_250')
xirr_te = summary_te['xirr_percent']
print('XIRR TE: {}%'.format(xirr_te))

XIRR TR: 32.14%
XIRR VA: 31.81%
XIRR TE: 26.97%


In [118]:
pd.Series(model.feature_importances_, index=features_to_model).sort_values(ascending=False).head(10)

book_value_per_share    0.278709
price_to_book           0.110211
price_to_sales          0.056247
earnings_yield          0.051998
intangible_assets       0.050876
current_ratio           0.049554
pe_pb_product           0.043826
eps                     0.043211
market_cap              0.033766
eps_diluted             0.020199
dtype: float64